In [2]:
import sys

In [3]:
sys.path.append('/home/dkoes/git/gnina-torch')

In [4]:
from gninatorch.gnina import setup_gnina_model
import molgrid
import torch
import json
import glob

In [5]:
ls gninatorch/weights

ls: cannot access 'gninatorch/weights': No such file or directory


In [6]:
d = {
    'resolution': 0.5,
    'dimension' : 23.5,
    'recmap' : '''AliphaticCarbonXSHydrophobe 
AliphaticCarbonXSNonHydrophobe 
AromaticCarbonXSHydrophobe 
AromaticCarbonXSNonHydrophobe
Bromine Iodine Chlorine Fluorine
Nitrogen NitrogenXSAcceptor 
NitrogenXSDonor NitrogenXSDonorAcceptor
Oxygen OxygenXSAcceptor 
OxygenXSDonorAcceptor OxygenXSDonor
Sulfur SulfurAcceptor
Phosphorus 
Calcium
Zinc
GenericMetal Boron Manganese Magnesium Iron''',
    'ligmap': '''AliphaticCarbonXSHydrophobe 
AliphaticCarbonXSNonHydrophobe 
AromaticCarbonXSHydrophobe 
AromaticCarbonXSNonHydrophobe
Bromine Iodine
Chlorine
Fluorine
Nitrogen NitrogenXSAcceptor 
NitrogenXSDonor NitrogenXSDonorAcceptor
Oxygen OxygenXSAcceptor 
OxygenXSDonorAcceptor OxygenXSDonor
Sulfur SulfurAcceptor
Phosphorus
GenericMetal Boron Manganese Magnesium Zinc Calcium Iron'''
}

old = {
        'resolution': 0.5,
    'dimension' : 23.5,
    'recmap' : '''AliphaticCarbonXSHydrophobe
AliphaticCarbonXSNonHydrophobe
AromaticCarbonXSHydrophobe
AromaticCarbonXSNonHydrophobe
Calcium
Iron
Magnesium
Nitrogen
NitrogenXSAcceptor
NitrogenXSDonor
NitrogenXSDonorAcceptor
OxygenXSAcceptor
OxygenXSDonorAcceptor
Phosphorus
Sulfur
Zinc''',
'ligmap':'''AliphaticCarbonXSHydrophobe
AliphaticCarbonXSNonHydrophobe
AromaticCarbonXSHydrophobe
AromaticCarbonXSNonHydrophobe
Bromine
Chlorine
Fluorine
Nitrogen
NitrogenXSAcceptor
NitrogenXSDonor
NitrogenXSDonorAcceptor
Oxygen
OxygenXSAcceptor
OxygenXSDonorAcceptor
Phosphorus
Sulfur
SulfurAcceptor
Iodine
Boron'''
}

In [7]:
for fname in glob.glob('gninatorch/weights/*.pt'):
    newf = fname.split('/')[-1]
    prefix = newf[:-3]
    
    model = setup_gnina_model(prefix)[0]
    model.eval()
    if 'default2017' in fname:
        extra = {'metadata':json.dumps(old)}
        z = torch.zeros((1,35,48,48,48))
    else:
        extra = {'metadata':json.dumps(d)}
        z = torch.zeros((1,28,48,48,48))
        
    script = torch.jit.trace(model, z)
    script.save(newf,_extra_files=extra)    

In [78]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Overlap(nn.Module):
    '''Compute overlap of single channel ligand and receptor'''
    def __init__(self):
        super().__init__()

    def forward(xelf, x):
        lig = x[:,1,:,:,:]
        rec = x[:,0,:,:,:]
        prot = rec * lig
        ave = F.avg_pool3d(prot,48).flatten(1)
        ave = torch.where(ave > 0, ave, 1e-20)
        cave = torch.hstack([torch.zeros_like(ave),ave])
        return (cave,torch.zeros((1,1),device=x.device))

In [79]:
x = torch.rand((1,2,48,48,48),requires_grad=True)

In [80]:
x.requires_grad = True

In [81]:
omodel = Overlap()

In [82]:
single = {
    'resolution': 0.5,
    'dimension' : 23.5,
    'skip_softmax' : True,    
    'apply_logistic_loss' : True,
    'recmap' : '''AliphaticCarbonXSHydrophobe AliphaticCarbonXSNonHydrophobe AromaticCarbonXSHydrophobe AromaticCarbonXSNonHydrophobe Bromine Iodine Chlorine Fluorine Nitrogen NitrogenXSAcceptor NitrogenXSDonor NitrogenXSDonorAcceptor Oxygen OxygenXSAcceptor OxygenXSDonorAcceptor OxygenXSDonor Sulfur SulfurAcceptor Phosphorus GenericMetal Boron Manganese Magnesium Zinc Calcium Iron''',
    'ligmap': '''AliphaticCarbonXSHydrophobe AliphaticCarbonXSNonHydrophobe AromaticCarbonXSHydrophobe AromaticCarbonXSNonHydrophobe Bromine Iodine Chlorine Fluorine Nitrogen NitrogenXSAcceptor NitrogenXSDonor NitrogenXSDonorAcceptor Oxygen OxygenXSAcceptor OxygenXSDonorAcceptor OxygenXSDonor Sulfur SulfurAcceptor Phosphorus GenericMetal Boron Manganese Magnesium Zinc Calcium Iron'''
}

In [83]:
oscript = torch.jit.trace(omodel, x)

In [84]:
oscript.save('data/overlap.pt',_extra_files={'metadata':json.dumps(single)})

In [85]:
smallr = {
    'resolution': 0.5,
    'dimension' : 23.5,
    'skip_softmax' : True,
    'apply_logistic_loss' : True,
    'radius_scaling' : 0.5,
    'recmap' : '''AliphaticCarbonXSHydrophobe AliphaticCarbonXSNonHydrophobe AromaticCarbonXSHydrophobe AromaticCarbonXSNonHydrophobe Bromine Iodine Chlorine Fluorine Nitrogen NitrogenXSAcceptor NitrogenXSDonor NitrogenXSDonorAcceptor Oxygen OxygenXSAcceptor OxygenXSDonorAcceptor OxygenXSDonor Sulfur SulfurAcceptor Phosphorus GenericMetal Boron Manganese Magnesium Zinc Calcium Iron''',
    'ligmap': '''AliphaticCarbonXSHydrophobe AliphaticCarbonXSNonHydrophobe AromaticCarbonXSHydrophobe AromaticCarbonXSNonHydrophobe Bromine Iodine Chlorine Fluorine Nitrogen NitrogenXSAcceptor NitrogenXSDonor NitrogenXSDonorAcceptor Oxygen OxygenXSAcceptor OxygenXSDonorAcceptor OxygenXSDonor Sulfur SulfurAcceptor Phosphorus GenericMetal Boron Manganese Magnesium Zinc Calcium Iron'''
}

In [86]:
oscript.save('data/overlap_smallr.pt',_extra_files={'metadata':json.dumps(smallr)})

In [59]:
torch.log(torch.Tensor([1e-10]))

tensor([-23.0259])